<a href="https://colab.research.google.com/github/Lasal0425/Machine-Learning-Tutorials/blob/main/Tute6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#To transform your features into new values
from sklearn import preprocessing

#Import train_test_split function
from sklearn.model_selection import train_test_split

#To construct and plot decission trees
from sklearn import tree
from matplotlib import pyplot as plt

#import Decision Tree Classifier to predict loan approval status
from sklearn.tree import DecisionTreeClassifier

#import Decision Tree Regressor to predict maximum loan amount
from sklearn.tree import DecisionTreeRegressor

#to optimise the Decision Tree model's hyperparamters
from sklearn.model_selection import GridSearchCV

#import scikiti-learn metrices module for classification and regression performance metrics
from sklearn import metrics

#to create and plot the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

#to produce the classification metrices report
from sklearn.metrics import classification_report

#to create, calculate and plot the RoC curve
from sklearn.metrics import RocCurveDisplay

In [5]:
# To expand the truncation of rows and columns
import pandas as pd # Import the pandas library and assign it to the alias 'pd'
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 150)

# To expand e scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)

In [6]:
data =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tutorial 06/loan_approval_data.csv')

In [7]:
data.shape

(58645, 13)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       58645 non-null  int64  
 1   age                      58639 non-null  float64
 2   income                   58645 non-null  int64  
 3   home_ownership           58645 non-null  object 
 4   emplyment_length         58645 non-null  int64  
 5   loan_intent              58645 non-null  object 
 6   loan_amount              58645 non-null  int64  
 7   loan_interest_rate       58634 non-null  float64
 8   loan_income_ratio        58645 non-null  float64
 9   payment_default_on_file  58640 non-null  object 
 10  credit_history_length    58645 non-null  int64  
 11  loan_approval_status     58645 non-null  int64  
 12  max_allowed_loan         58645 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 5.8+ MB


In [9]:
data.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,58645.00,NaN,NaN,NaN,29322.00,16929.50,0.00,14661.00,29322.00,43983.00,58644.00
age,58639.00,NaN,NaN,NaN,27.55,6.03,20.00,23.00,26.00,30.00,123.00
income,58645.00,NaN,NaN,NaN,64046.17,37931.11,4200.00,42000.00,58000.00,75600.00,1900000.00
home_ownership,58645,4,RENT,30594,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emplyment_length,58645.00,NaN,NaN,NaN,4.70,4.00,0.00,2.00,4.00,7.00,150.00
loan_intent,58645,6,EDUCATION,12271,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amount,58645.00,NaN,NaN,NaN,9217.56,5563.81,500.00,5000.00,8000.00,12000.00,35000.00
loan_interest_rate,58634.00,NaN,NaN,NaN,10.68,3.04,-11.14,7.88,10.75,12.99,23.22
loan_income_ratio,58645.00,NaN,NaN,NaN,0.16,0.09,0.00,0.09,0.14,0.21,0.83
payment_default_on_file,58640,2,N,49940,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
!pip install plotly
import plotly.express as px # Importing the plotly.express library and assigning it to the alias 'px'
import pandas as pd # Importing the pandas library and assigning it to the alias 'pd'

# ... (rest of your code) ...

loan_approval_status_fig = px.histogram(data, x='loan_approval_status')
loan_max_amount_fig = px.histogram(data, x='max_allowed_loan')
loan_approval_status_fig.show()
loan_max_amount_fig.show()

In [12]:
data.isna().sum()/len(data)*100

,0
id,0.00
age,0.01
income,0.00
home_ownership,0.00
emplyment_length,0.00
loan_intent,0.00
loan_amount,0.00
loan_interest_rate,0.02
loan_income_ratio,0.00
payment_default_on_file,0.01


In [13]:
print('home_ownership categories are:', data['home_ownership'].unique())
print('loan_intent categories are:',data['loan_intent'].unique())
print('payment_default_on_file categories are:',data['payment_default_on_file'].unique())

home_ownership categories are: ['OWN' 'RENT' 'MORTGAGE' 'OTHER']
loan_intent categories are: ['EDUCATION' 'MEDICAL' 'HOMEIMPROVEMENT' 'VENTURE' 'DEBTCONSOLIDATION'
 'PERSONAL']
payment_default_on_file categories are: ['N' 'Y' nan]


In [14]:
#Create a function to find the outlier data points
def find_outliers_IQR(data):
  q1=data.quantile(0.25)
  q3=data.quantile(0.75)
  IQR=q3-q1
  outliers = data[((data<(q1-1.5*IQR)) | (data>(q3+1.5*IQR)))]
  return outliers

In [15]:
# Find the outlier data points in the age variable
age_outliers = find_outliers_IQR(data['age'])
print("number of outliers: "+ str(len(age_outliers)))
print("max outlier value: "+ str(age_outliers.max()))
print("min outlier value: "+ str(age_outliers.min()))
age_outliers.sort_values(ascending=False)

number of outliers: 2446
max outlier value: 123.0
min outlier value: 41.0


,age
39792,123.00
56540,84.00
54600,84.00
8728,80.00
8727,80.00
8726,76.00
25687,73.00
8725,73.00
15263,73.00
36007,70.00


In [16]:
# Find the outlier data points in emplyment_length variable
emplyment_length_outliers = find_outliers_IQR(data['emplyment_length'])
print("number of outliers: "+ str(len(emplyment_length_outliers)))
print("max outlier value: "+ str(emplyment_length_outliers.max()))
print("min outlier value: "+ str(emplyment_length_outliers.min()))
emplyment_length_outliers.sort_values(ascending=False)

number of outliers: 1275
max outlier value: 150
min outlier value: 15


,emplyment_length
9479,150
6008,123
56728,123
37054,41
44566,41
48831,39
37571,35
51756,31
8573,31
33088,31


In [17]:
# Find the outlier data points in loan_interest_rate variable
loan_interest_rate_outliers = find_outliers_IQR(data['loan_interest_rate'])
print("number of outliers: "+ str(len(loan_interest_rate_outliers)))
print("max outlier value: "+ str(loan_interest_rate_outliers.max()))
print("min outlier value: "+ str(loan_interest_rate_outliers.min()))
loan_interest_rate_outliers.sort_values(ascending=True)

number of outliers: 35
max outlier value: 23.22
min outlier value: -11.14


,loan_interest_rate
8998,-11.14
10845,20.69
50909,20.69
3198,20.69
4234,20.80
4232,20.86
2837,20.86
35364,20.89
6303,20.89
2842,20.89


In [18]:
# Find the outlier data points in max_allowed_loan variable
max_allowed_loan_outliers = find_outliers_IQR(data['max_allowed_loan'])
print("number of outliers: "+ str(len(max_allowed_loan_outliers)))
print("max outlier value: "+ str(max_allowed_loan_outliers.max()))
print("min outlier value: "+ str(max_allowed_loan_outliers.min()))
max_allowed_loan_outliers.sort_values(ascending=True)

number of outliers: 2233
max outlier value: 2638778
min outlier value: -2426900


,max_allowed_loan
0,-2426900
1,-111739
2,-89000
56415,174791
56416,174817
56417,174945
56418,174953
56419,174953
56420,174958
56421,174963


In [19]:
data.drop(data.index[[39792, 9479, 6008, 56728, 8998, 0, 1, 2]], inplace=True)

In [20]:
age_fig = px.box(data, x='age')
emplyment_length_fig = px.box(data, x='emplyment_length')
loan_interest_rate_fig = px.box(data, x='loan_interest_rate')
max_allowed_loan_fig = px.box(data, x='max_allowed_loan')

age_fig.show()
emplyment_length_fig.show()
loan_interest_rate_fig.show()
max_allowed_loan_fig.show()

In [21]:
# Import label encoder
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

# Encode the categories in each object variable to a numeric form
data['home_ownership']= label_encoder.fit_transform(data['home_ownership'])
data['loan_intent']= label_encoder.fit_transform(data['loan_intent'])
data['payment_default_on_file']= label_encoder.fit_transform(data['payment_default_on_file'])

# Check the categories' labels after the label encoding transformation is completed
print("New labels for home_ownership are: ", data['home_ownership'].unique())
print("New labels for loan_intent are: ", data['loan_intent'].unique())
print("New labels for payment_default_on_file are: ", data['payment_default_on_file'].unique())

New labels for home_ownership are:  [3 0 2 1]
New labels for loan_intent are:  [1 2 5 0 4 3]
New labels for payment_default_on_file are:  [0 1 2]


In [22]:
# Calculate the mean of each variable
Mean_age = data['age'].mean()
Mean_loan_interest_rate = data['loan_interest_rate'].mean()

# use the mean to impute the missing values in each variable
data['age'].fillna(Mean_age, inplace=True)
data['loan_interest_rate'].fillna(Mean_loan_interest_rate, inplace=True)

<ipython-input-22-42a0238689cf>:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



<ipython-input-22-42a0238689cf>:7: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[

In [23]:
data.isna().sum()/len(data)*100

,0
id,0.00
age,0.00
income,0.00
home_ownership,0.00
emplyment_length,0.00
loan_intent,0.00
loan_amount,0.00
loan_interest_rate,0.00
loan_income_ratio,0.00
payment_default_on_file,0.00


In [25]:
data.to_csv(r'/content/prepared_loan_approval_data.csv', index=False)

In [26]:
#Load your prepared dataset as df
df = pd.read_csv('/content/prepared_loan_approval_data.csv')

# Keep the target for classification and drop only the regression target max_allowed_loan
df.drop('max_allowed_loan',axis=1, inplace=True)
df.head()

,id,age,income,home_ownership,emplyment_length,loan_intent,loan_amount,loan_interest_rate,loan_income_ratio,payment_default_on_file,credit_history_length,loan_approval_status
0,19180,40.00,182004,3,3,1,35000,8.00,0.19,0,11,0
1,28072,40.00,90000,0,3,2,35000,12.42,0.39,0,14,0
2,32074,40.00,131004,0,9,5,30000,7.90,0.23,0,13,0
3,7224,40.00,150000,0,9,0,25000,11.89,0.17,0,11,0
4,14471,40.00,132000,0,3,2,25000,16.82,0.22,0,17,0


In [27]:
#Save your classification dataset under the name loan_approval_status_data
df.to_csv(r'/content/loan_approval_status_data.csv', index=False)

In [28]:
#load your fully prepared dataset as df2
df2=pd.read_csv('/content/prepared_loan_approval_data.csv')

# Filter approved applicants (loan_approval_status for status = 0) in a new data frame called approved_loan_applicants
approved_loan_applicants =  df2[(df2.loan_approval_status < 1)]

# Check the basic stats to observe only approved applicants remain
approved_loan_applicants.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,50288.00,29377.31,16930.54,0.00,14734.75,29396.50,44020.25,58644.00
age,50288.00,27.55,5.96,20.00,23.00,26.00,30.00,84.00
income,50288.00,66675.95,39309.70,4200.00,45000.00,60000.00,80000.00,1900000.00
home_ownership,50288.00,1.54,1.46,0.00,0.00,2.00,3.00,3.00
emplyment_length,50288.00,4.86,3.91,0.00,2.00,4.00,7.00,41.00
loan_intent,50288.00,2.55,1.73,0.00,1.00,3.00,4.00,5.00
loan_amount,50288.00,8888.05,5326.99,500.00,5000.00,8000.00,12000.00,35000.00
loan_interest_rate,50288.00,10.26,2.83,5.42,7.51,10.39,12.42,22.11
loan_income_ratio,50288.00,0.15,0.08,0.00,0.08,0.13,0.20,0.63
payment_default_on_file,50288.00,0.12,0.33,0.00,0.00,0.00,0.00,2.00


In [29]:
# Drop the loan_approval_status variable from the data frame
approved_loan_applicants.drop('loan_approval_status',axis=1, inplace=True)
approved_loan_applicants.head()

<ipython-input-29-f5471eed7f8c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,age,income,home_ownership,emplyment_length,loan_intent,loan_amount,loan_interest_rate,loan_income_ratio,payment_default_on_file,credit_history_length,max_allowed_loan
0,19180,40.00,182004,3,3,1,35000,8.00,0.19,0,11,35000
1,28072,40.00,90000,0,3,2,35000,12.42,0.39,0,14,35000
2,32074,40.00,131004,0,9,5,30000,7.90,0.23,0,13,30000
3,7224,40.00,150000,0,9,0,25000,11.89,0.17,0,11,25000
4,14471,40.00,132000,0,3,2,25000,16.82,0.22,0,17,25000


In [30]:
approved_loan_applicants.to_csv(r'/content/loan_max_amount_data.csv', index=False)